In [ ]:
import os
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import uuid
from pathlib import Path
import os.path
from os import path

In [ ]:
def get_modified_date(source_file, exiftool_file_path):

    command = "exiftool -time:all -a -G0:1 -s " + source_file + " > " + exiftool_file_path
    os.system(command)

    file_modified_date = ""
    if path.exists(exiftool_file_path) == False:
        return file_modified_date
    
    with open(exiftool_file_path, 'r') as fh:
        for line in fh:
            line_info = line.split(":", 2)
            if len(line_info) >= 2:
                metadata = line_info[1]
                metadata_value = line_info[2]
                if "FileModifyDate" in metadata: 
                    file_modified_date = metadata_value
                    break
    return file_modified_date

In [ ]:
folder = "2003"
folder_path = "/data/" + folder
fits_sh = "/home/fits/fits.sh"
metadata_file_output_folder = "/home/fits_output/"
exiftool_output_folder = "/home/exiftool_output/"
reports_folder = "/home/notebooks/reports/"

In [ ]:
files = []
files_metadata_df = pd.DataFrame()

for root,d_names,f_names in os.walk(folder_path):
    for f in f_names:
        files.append(os.path.join(root, f))
        
for file in files:
    print("processing " + file)
    head, file_name = os.path.split(file)
    uuid_str = str(uuid.uuid4())
    fits_file_name = uuid_str + ".xml"
    exiftool_file_path = exiftool_output_folder + uuid_str + ".txt"
    metadata_file = metadata_file_output_folder + fits_file_name 

    path_info = file.replace(folder_path + "/", "")
    path_info_list = path_info.split("/")
    path_info_list = path_info_list[:-1]
    path_info_str = '|'.join(path_info_list)    
    path_info_str = folder + "|" + path_info_str
    
    file = file.replace("'", r"\'")
    file = file.replace('"', r'\"')
    file = file.replace("&", r"\&")
    file = file.replace("(", r"\(")
    file = file.replace(")", r"\)")
    file = file.replace(",", r"\,")
    
    command = fits_sh + " -i " +  file + " -o " + metadata_file
    os.system(command)
    
    file_modify_date = get_modified_date(file, exiftool_file_path)

    md5_checksum = ""
    mime_type = ""        
    created_date = ""
    copy_rights = ""
    software_agent_1 = ""
    software_agent_2 = ""
    keywords = ""
    
    if path.exists(metadata_file) == False:
        files_metadata_df = files_metadata_df.append({'original_path': file, 'file_name': file_name, 'md5_checksum':md5_checksum, 'mime_type': mime_type, 'path_info': path_info_str, 'created_date': created_date, 'copy_rights': copy_rights, 'software_agent_1': software_agent_1, 'software_agent_2': software_agent_2, 'keywords': keywords, 'fits_file_name': fits_file_name, 'file_modify_date': file_modify_date}, ignore_index=True)
        continue

    with open(metadata_file) as fp:
        file_xml = BeautifulSoup(fp, "xml")    
        fits_info = file_xml.findAll("fits")[0]
        md5_checksum = fits_info.find("fileinfo").find("md5checksum").text.strip()
        exiftool_output = fits_info.find("tool", {"name" : "Exiftool"})

        if exiftool_output is not None:
            
            mime_type_node = exiftool_output.find("MIMEType")
            if mime_type_node is not None:
                mime_type = mime_type_node.text.strip()
                    
            created_date_node = exiftool_output.find("CreateDate")
            if created_date_node is not None:
                created_date = created_date_node.text.strip()

            if created_date == "":
                profile_date_time = exiftool_output.find("ProfileDateTime")
                if profile_date_time is not None:
                    created_date = profile_date_time.text.strip()
                        
            copy_right_notice = exiftool_output.find("CopyrightNotice")
            if copy_right_notice is not None:
                copy_rights = copy_right_notice.text.strip()
            
            history_software_agent = exiftool_output.find("HistorySoftwareAgent")
            if history_software_agent is not None:
                software_agent_1 = history_software_agent.text.strip()
            
            profile_creator = exiftool_output.find("ProfileCreator")
            if profile_creator is not None:
                software_agent_2 = profile_creator.text.strip()
            
            keywords_node = exiftool_output.find("Keywords")
            if keywords_node is not None:
                keywords = keywords_node.text.strip()

        files_metadata_df = files_metadata_df.append({'original_path': file, 'file_name': file_name, 'md5_checksum':md5_checksum, 'mime_type': mime_type, 'path_info': path_info_str, 'created_date': created_date, 'copy_rights': copy_rights, 'software_agent_1': software_agent_1, 'software_agent_2': software_agent_2, 'keywords': keywords, 'fits_file_name': fits_file_name, 'file_modify_date': file_modify_date}, ignore_index=True)

In [ ]:
files_metadata_df = files_metadata_df[['original_path',"file_name","md5_checksum","mime_type", 'path_info', 'created_date', 'file_modify_date', 'keywords', 'copy_rights', 'software_agent_1', 'software_agent_2', 'fits_file_name']]
files_metadata_df

In [ ]:
files_metadata_df.to_csv(reports_folder + folder + ".csv")

In [ ]:
files_metadata_df